In [0]:
catalog_name = dbutils.widgets.text("catalog_name", "")
catalog_name = dbutils.widgets.get("catalog_name")
bronze_schema = dbutils.widgets.text("bronze_schema", "")
bronze_schema = dbutils.widgets.get("bronze_schema")
silver_schema = dbutils.widgets.text("silver_schema", "")
silver_schema = dbutils.widgets.get("silver_schema")
gold_schema = dbutils.widgets.text("gold_schema", "")
gold_schema = dbutils.widgets.get("gold_schema")
input_table_g_client = dbutils.widgets.text("input_table_g_client", "")
input_table_g_client = dbutils.widgets.get("input_table_g_client")
output_table_g_client = dbutils.widgets.text("output_table_g_client", "")
output_table_g_client = dbutils.widgets.get("output_table_g_client")

In [0]:

from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import col, trim, initcap, when, lit, year, month, dayofmonth, concat, concat_ws
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException
import time

In [0]:
schema_auditoria ="auditoria"

In [0]:
sql_gold = f"""
    CREATE SCHEMA IF NOT EXISTS {catalog_name}.{gold_schema}
"""

spark.sql(sql_gold)

In [0]:
gold_dim_cliente = (
    spark.table(f"{catalog_name}.{silver_schema}.{input_table_g_client}")
    .select("id_cliente", "nombre", "email", "ciudad", "fecha_registro")
    .withColumn("creation_date", F.current_timestamp())
    .dropDuplicates(["id_cliente"])
)


In [0]:
display(gold_dim_cliente)

In [0]:
# Crear tabla destino
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.{gold_schema}.{output_table_g_client} (
  id_cliente BIGINT,
  nombre STRING,
  email STRING,
  ciudad STRING,
  fecha_registro DATE,
  creation_date TIMESTAMP
) USING DELTA
""")

In [0]:

delta_target = DeltaTable.forName(spark, f"{catalog_name}.{gold_schema}.{output_table_g_client}")

merge_start = time.time()
delta_target.alias("tgt") \
  .merge(
    source=gold_dim_cliente.alias("src"),
    condition="tgt.id_cliente = src.id_cliente"
  ) \
  .whenMatchedUpdateAll(
    condition="src.creation_date > tgt.creation_date"
  ) \
  .whenNotMatchedInsertAll() \
  .execute()
merge_end = time.time()

In [0]:
spark.table(f"{catalog_name}.{gold_schema}.{output_table_g_client}").show(10)

In [0]:
import time
job_id = int(time.time())           # ID único basado en timestamp
job_run_id = int(time.time() * 100) # más granular
task_run_id = 1                     # o secuencial si prefieres


In [0]:
rows_in = gold_dim_cliente.count()
start = time.time()
spark.sql(f"""
INSERT INTO {catalog_name}.{schema_auditoria}.ingestion_log VALUES (
  {job_id},{job_run_id},{task_run_id},
  current_timestamp(), current_timestamp(),
  {int(time.time()-start)},
  'SUCCESS','dim_clientes','gold',
  {rows_in},{rows_in},0,0,0,{int(merge_end-merge_start)},current_timestamp()
)
""")

In [0]:
display(spark.table(f"{catalog_name}.{schema_auditoria}.ingestion_log"))


In [0]:

sql_command = f"""
  OPTIMIZE {catalog_name}.{gold_schema}.{output_table_g_client}
  ZORDER BY (id_cliente)
"""

In [0]:
spark.sql(sql_command)


In [0]:
spark.sql(f"VACUUM {catalog_name}.{gold_schema}.{output_table_g_client}")